In [1]:
import os 

In [2]:
%pwd

'c:\\ML_OPS\\Projects\\project1\\research'

In [3]:
os.chdir('../')

In [8]:
from src.DataScienceProject.config import *
from src.DataScienceProject import logger
from dataclasses import dataclass
from pathlib import Path
from src.DataScienceProject.constants import *


In [10]:
from src.DataScienceProject.utils.shared import read_yaml,create_directories

In [11]:
@dataclass
class ModelTrainingConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: str
    l1_ratio: str
    target_column: str
    

In [12]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH
                 ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)


        create_directories([self.config.artifacts_root])
    def get_model_trainer_config(self) -> ModelTrainingConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        create_directories([config.root_dir])
        model_trainer_config= ModelTrainingConfig(
            root_dir= config.root_dir,
            train_data_path= config.train_data_path,
            test_data_path= config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio= params.l1ratio,
            target_column = schema.name
        )
        return model_trainer_config

        

In [13]:
import pandas as pd 
from sklearn.linear_model import ElasticNet
import joblib # Alternative of pickle


In [ ]:
class ModelTrainer:
    def __init__(self,config: ModelTrainingConfig):
        self.config = config
        # self.model = ElasticNet(alpha=config.alpha, l1_ratio=config.l1_ratio)
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop(columns=[self.config.target_column],axis=1)
        test_x = test_data.drop(columns=[self.config.target_column],axis=1)

        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio)
        lr.fit(train_x, train_y)

        joblib.dump(lr,os.path.join(self.config.root_dir,self.config.model_name))